# 21장. 신경망

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/ml-with-python-cookbook-2nd/blob/main/ch21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

In [ ]:
import numpy as np
import sklearn
import torch
import matplotlib
import lightening
import ray
import torchvis

print('numpy', np.__version__)
print('sklearn', sklearn.__version__)
print('torch', torch.__version__)
print('matplotlib', matplotlib.__version__)
print('lightening', lightening.__version__)
print('ray', ray.__version__)
print('torchvis', torchvis.__version__)

## 21.1 파이토치 자동미분 사용하기

In [1]:
# 라이브러리를 임포트합니다.
import torch

# 그레이언트가 필요한 토치 텐서를 만듭니다.
t = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

# 정방향 계산을 모방한 텐서 연산을 수행합니다.
tensor_sum = t.sum()

# 역전파를 수행합니다.
tensor_sum.backward()

# 그레이디언트를 확인합니다.
t.grad

ModuleNotFoundError: No module named 'torch'

In [ ]:
import torch

tensor = torch.tensor([1.0,2.0,3.0], requires_grad=True)
tensor.numpy()

## 21.2 신경망을 위해 데이터 전처리하기

In [ ]:
# 라이브러리를 임포트합니다.
from sklearn import preprocessing
import numpy as np

# 특성을 만듭니다.
features = np.array([[-100.1, 3240.1],
                     [-200.2, -234.1],
                     [5000.5, 150.1],
                     [6000.6, -125.1],
                     [9000.9, -673.1]])

# 스케일링 객체를 만듭니다.
scaler = preprocessing.StandardScaler()

# 특성을 변환합니다.
features_standardized = scaler.fit_transform(features)

# 특성을 확인합니다.
features_standardized

In [ ]:
# 라이브러리를 임포트합니다.
import torch

# 특성을 만듭니다.
torch_features = torch.tensor([[-100.1, 3240.1],
                               [-200.2, -234.1],
                               [5000.5, 150.1],
                               [6000.6, -125.1],
                               [9000.9, -673.1]], requires_grad=True)

# 평균과 표준 편차를 계산합니다.
mean = torch_features.mean(0, keepdim=True)
standard_deviation = torch_features.std(0, unbiased=False, keepdim=True)

# 평균과 표준 편차를 사용해 특성을 표준화합니다.
torch_features_standardized = torch_features - mean
torch_features_standardized /= standard_deviation

# 표준화된 특성을 확인합니다.
torch_features_standardized

## 21.3 신경망 구성하기

In [2]:
# 라이브러리를 임포트합니다.
import torch
import torch.nn as nn

# 신경망을 정의합니다.
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet, self).__init__()
        self.fc1 = nn.Linear(10, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 1)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.sigmoid(self.fc3(x))
        return x

# 신경망을 초기화합니다.
network = SimpleNeuralNet()

# 손실 함수, 옵티마이저를 정의합니다.
loss_criterion = nn.BCELoss()
optimizer = torch.optim.RMSprop(network.parameters())

# 신경망을 확인합니다.
network

ModuleNotFoundError: No module named 'torch'

In [ ]:
# 라이브러리를 임포트합니다.
import torch

# Sequential 클래스를 사용해 신경망을 정의합니다.
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet, self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(10, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.sequential(x)
        return x

# 신경망을 초기화하고 확인합니다.
SimpleNeuralNet()

## 21.4 이진 분류기 훈련하기

In [ ]:
# 라이브러리를 임포트합니다.
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import RMSprop
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# 훈련 세트와 테스트 세트를 만듭니다.
features, target = make_classification(n_classes=2, n_features=10,
    n_samples=1000)
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.1, random_state=1)

# 랜덤 시드를 지정합니다.
torch.manual_seed(0)
np.random.seed(0)

# 데이터를 파이토치 텐서로 변환합니다.
x_train = torch.from_numpy(features_train).float()
y_train = torch.from_numpy(target_train).float().view(-1, 1)
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(target_test).float().view(-1, 1)

# Sequential 클래스를 사용해 신경망을 정의합니다.
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet, self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(10, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.sequential(x)
        return x

# 신경망을 초기화합니다.
network = SimpleNeuralNet()

# 손실 함수와 옵티마이저를 정의합니다.
criterion = nn.BCELoss()
optimizer = RMSprop(network.parameters())

# 데이터 로더를 정의합니다.
train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=100, shuffle=True)

# 파이토치 2.0의 옵티마이저를 사용해 모델을 컴파일합니다.
network = torch.compile(network)

# 신경망을 훈련합니다.
epochs = 3
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print("에포크:", epoch+1, "\t손실:", loss.item())

# 신경망을 평가합니다.
with torch.no_grad():
    output = network(x_test)
    test_loss = criterion(output, y_test)
    test_accuracy = (output.round() == y_test).float().mean()
    print("테스트 세트 손실:", test_loss.item(), "\t테스트 세트 정확도:",
        test_accuracy.item())

## 21.5 다중 분류기 훈련하기

In [ ]:
# 라이브러리를 임포트합니다.
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import RMSprop
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

N_CLASSES=3
EPOCHS=3

# 훈련 세트와 테스트 세트를 만듭니다.
features, target = make_classification(n_classes=N_CLASSES, n_informative=9,
    n_redundant=0, n_features=10, n_samples=1000)
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.1, random_state=1)

# 랜덤 시드를 지정합니다.
torch.manual_seed(0)
np.random.seed(0)

# 데이터를 파이토치 텐서로 변환합니다.
x_train = torch.from_numpy(features_train).float()
y_train = torch.nn.functional.one_hot(torch.from_numpy(target_train).long(),
    num_classes=N_CLASSES).float()
x_test = torch.from_numpy(features_test).float()
y_test = torch.nn.functional.one_hot(torch.from_numpy(target_test).long(),
    num_classes=N_CLASSES).float()

# Sequential 클래스를 사용해 신경망을 정의합니다.
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet, self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(10, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,3),
            torch.nn.Softmax()
        )

    def forward(self, x):
        x = self.sequential(x)
        return x

# 신경망을 초기화합니다.
network = SimpleNeuralNet()

# 손실 함수와 옵티마이저를 정의합니다.
criterion = nn.CrossEntropyLoss()
optimizer = RMSprop(network.parameters())

# 데이터 로더를 정의합니다.
train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=100, shuffle=True)

# 모델을 컴파일합니다.
network = torch.compile(network)

# 신경망을 훈련합니다.
for epoch in range(EPOCHS):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print("에포크:", epoch+1, "\t손실:", loss.item())

# Evaluate neural network
with torch.no_grad():
    output = network(x_test)
    test_loss = criterion(output, y_test)
    test_accuracy = (output.round() == y_test).float().mean()
    print("테스트 세트 손실:", test_loss.item(), "\t테스트 세트 정확도:",
        test_accuracy.item())

In [ ]:
# 타깃 행렬을 확인합니다.
y_train

## 21.6 회귀 모델 훈련하기

In [ ]:
# 라이브러리를 임포트합니다.
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import RMSprop
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

EPOCHS=5

# 훈련 세트와 테스트 세트를 만듭니다.
features, target = make_regression(n_features=10, n_samples=1000)
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.1, random_state=1)

# 랜덤 시드를 지정합니다.
torch.manual_seed(0)
np.random.seed(0)

# 데이터를 파이토치 텐서로 변환합니다.
x_train = torch.from_numpy(features_train).float()
y_train = torch.from_numpy(target_train).float().view(-1,1)
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(target_test).float().view(-1,1)

# Sequential 클래스를 사용해 신경망을 정의합니다.
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet, self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(10, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,1),
        )

    def forward(self, x):
        x = self.sequential(x)
        return x

# 신경망을 초기화합니다.
network = SimpleNeuralNet()

# 손실 함수와 옵티마이저를 정의합니다.
criterion = nn.MSELoss()
optimizer = RMSprop(network.parameters())

# 데이터 로더를 정의합니다.
train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=100, shuffle=True)

# 모델을 컴파일합니다.
network = torch.compile(network)

# 신경망을 훈련합니다.
for epoch in range(EPOCHS):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print("에포크:", epoch+1, "\t손실:", loss.item())

# 신경망을 평가합니다.
with torch.no_grad():
    output = network(x_test)
    test_loss = float(criterion(output, y_test))
    print("테스트 세트 MSE:", test_loss)

## 21.7 예측하기

In [ ]:
# 라이브러리를 임포트합니다.
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import RMSprop
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# 훈련 세트와 테스트 세트를 만듭니다.
features, target = make_classification(n_classes=2, n_features=10,
    n_samples=1000)
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.1, random_state=1)

# 랜덤 시드를 지정합니다.
torch.manual_seed(0)
np.random.seed(0)

# 데이터를 파이토치 텐서로 변환합니다.
x_train = torch.from_numpy(features_train).float()
y_train = torch.from_numpy(target_train).float().view(-1, 1)
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(target_test).float().view(-1, 1)

# Sequential 클래스를 사용해 신경망을 정의합니다.
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet, self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(10, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.sequential(x)
        return x

# 신경망을 초기화합니다.
network = SimpleNeuralNet()

# 손실 함수와 옵티마이저를 정의합니다.
criterion = nn.BCELoss()
optimizer = RMSprop(network.parameters())

# 데이터 로더를 정의합니다.
train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=100, shuffle=True)

# 모델을 컴파일합니다.
network = torch.compile(network)

# 신경망을 훈련합니다.
epochs = 3
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print("에포크:", epoch+1, "\t손실:", loss.item())

# 예측을 만듭니다.
with torch.no_grad():
    predicted_class = network.forward(x_train).round()

predicted_class[0]

## 21.8 훈련 기록 시각화하기

In [ ]:
# 라이브러리를 임포트합니다.
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import RMSprop
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

# 훈련 세트와 테스트 세트를 만듭니다.
features, target = make_classification(n_classes=2, n_features=10,
    n_samples=1000)
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.1, random_state=1)

# 랜덤 시드를 지정합니다.
torch.manual_seed(0)
np.random.seed(0)

# 데이터를 파이토치 텐서로 변환합니다.
x_train = torch.from_numpy(features_train).float()
y_train = torch.from_numpy(target_train).float().view(-1, 1)
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(target_test).float().view(-1, 1)

# Sequential 클래스를 사용해 신경망을 정의합니다.
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet, self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(10, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.sequential(x)
        return x

# 신경망을 초기화합니다.
network = SimpleNeuralNet()

# 손실 함수와 옵티마이저를 정의합니다.
criterion = nn.BCELoss()
optimizer = RMSprop(network.parameters())

# 데이터 로더를 정의합니다.
train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=100, shuffle=True)

# 모델을 컴파일합니다.
network = torch.compile(network)

# 신경망을 훈련합니다.
epochs = 8
train_losses = []
test_losses = []
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        train_output = network(x_train)
        train_loss = criterion(output, target)
        train_losses.append(train_loss.item())

        test_output = network(x_test)
        test_loss = criterion(test_output, y_test)
        test_losses.append(test_loss.item())

# 손실 기록을 시각화합니다.
epochs = range(0, epochs)
plt.plot(epochs, train_losses, "r--")
plt.plot(epochs, test_losses, "b-")
plt.legend(["Training Loss", "Test Loss"])
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show();

## 21.9 가중치 규제로 과대적합 줄이기

In [ ]:
# 라이브러리를 임포트합니다.
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import RMSprop
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# 훈련 세트와 테스트 세트를 만듭니다.
features, target = make_classification(n_classes=2, n_features=10,
    n_samples=1000)
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.1, random_state=1)

# 랜덤 시드를 지정합니다.
torch.manual_seed(0)
np.random.seed(0)

# 데이터를 파이토치 텐서로 변환합니다.
x_train = torch.from_numpy(features_train).float()
y_train = torch.from_numpy(target_train).float().view(-1, 1)
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(target_test).float().view(-1, 1)

# Sequential 클래스를 사용해 신경망을 정의합니다.
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet, self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(10, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.sequential(x)
        return x

# 신경망을 초기화합니다.
network = SimpleNeuralNet()

# 손실 함수와 옵티마이저를 정의합니다.
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(network.parameters(), lr=1e-4, weight_decay=1e-5)

# 데이터 로더를 정의합니다.
train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=100, shuffle=True)

# 모델을 컴파일합니다.
network = torch.compile(network)

# 신경망을 훈련합니다.
epochs = 100
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

# 신경망을 평가합니다.
with torch.no_grad():
    output = network(x_test)
    test_loss = criterion(output, y_test)
    test_accuracy = (output.round() == y_test).float().mean()
    print("테스트 세트 손실:", test_loss.item(), "\t테스트 세트 정확도:",
        test_accuracy.item())

## 21.10 조기종료로 과대적합 줄이기

In [ ]:
# 라이브러리를 임포트합니다.
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import RMSprop
import lightning as pl
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# 훈련 세트와 테스트 세트를 만듭니다.
features, target = make_classification(n_classes=2, n_features=10,
    n_samples=1000)
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.1, random_state=1)

# 랜덤 시드를 지정합니다.
torch.manual_seed(0)
np.random.seed(0)

# 데이터를 파이토치 텐서로 변환합니다.
x_train = torch.from_numpy(features_train).float()
y_train = torch.from_numpy(target_train).float().view(-1, 1)
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(target_test).float().view(-1, 1)

# Sequential 클래스를 사용해 신경망을 정의합니다.
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet, self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(10, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.sequential(x)
        return x

class LightningNetwork(pl.LightningModule):
    def __init__(self, network):
        super().__init__()
        self.network = network
        self.criterion = nn.BCELoss()
        self.metric = nn.functional.binary_cross_entropy

    def training_step(self, batch, batch_idx):
        # training_step는 훈련 루프를 정의합니다.
        data, target = batch
        output = self.network(data)
        loss = self.criterion(output, target)
        self.log("val_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

# 데이터 로더를 정의합니다.
train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=100, shuffle=True)

# 신경망을 초기화합니다.
network = LightningNetwork(SimpleNeuralNet())

# 신경망을 훈련합니다.
trainer = pl.Trainer(callbacks=[EarlyStopping(monitor="val_loss", mode="min",
    patience=3)], max_epochs=1000)
trainer.fit(model=network, train_dataloaders=train_loader)

In [ ]:
# 신경망을 훈련합니다.
trainer = pl.Trainer(max_epochs=1000)
trainer.fit(model=network, train_dataloaders=train_loader)

## 21.11 드롭아웃으로 과대적합 줄이기

In [ ]:
# 라이브러리를 임포트합니다.
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import RMSprop
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# 훈련 세트와 테스트 세트를 만듭니다.
features, target = make_classification(n_classes=2, n_features=10,
    n_samples=1000)
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.1, random_state=1)

# 랜덤 시드를 지정합니다.
torch.manual_seed(0)
np.random.seed(0)

# 데이터를 파이토치 텐서로 변환합니다.
x_train = torch.from_numpy(features_train).float()
y_train = torch.from_numpy(target_train).float().view(-1, 1)
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(target_test).float().view(-1, 1)

# Sequential 클래스를 사용해 신경망을 정의합니다.
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet, self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(10, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 1),
            torch.nn.Dropout(0.1), # Drop 10% of neurons
            torch.nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.sequential(x)
        return x

# 신경망을 초기화합니다.
network = SimpleNeuralNet()

# 손실 함수와 옵티마이저를 정의합니다.
criterion = nn.BCELoss()
optimizer = RMSprop(network.parameters())

# 데이터 로더를 정의합니다.
train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=100, shuffle=True)

# 모델을 컴파일합니다.
network = torch.compile(network)

# 신경망을 훈련합니다.
epochs = 3
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print("에포크:", epoch+1, "\t손실:", loss.item())

# 신경망을 평가합니다.
with torch.no_grad():
    output = network(x_test)
    test_loss = criterion(output, y_test)
    test_accuracy = (output.round() == y_test).float().mean()
    print("테스트 세트 손실:", test_loss.item(), "\t테스트 세트 정확도:",
        test_accuracy.item())

## 21.12 모델 훈련 진행 과정을 저장하기

In [ ]:
# 라이브러리를 임포트합니다.
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import RMSprop
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# 훈련 세트와 테스트 세트를 만듭니다.
features, target = make_classification(n_classes=2, n_features=10,
    n_samples=1000)
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.1, random_state=1)

# 랜덤 시드를 지정합니다.
torch.manual_seed(0)
np.random.seed(0)

# 데이터를 파이토치 텐서로 변환합니다.
x_train = torch.from_numpy(features_train).float()
y_train = torch.from_numpy(target_train).float().view(-1, 1)
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(target_test).float().view(-1, 1)

# Sequential 클래스를 사용해 신경망을 정의합니다.
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet, self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(10, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 1),
            torch.nn.Dropout(0.1), # Drop 10% of neurons
            torch.nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.sequential(x)
        return x

# 신경망을 초기화합니다.
network = SimpleNeuralNet()

# 손실 함수와 옵티마이저를 정의합니다.
criterion = nn.BCELoss()
optimizer = RMSprop(network.parameters())

# 데이터 로더를 정의합니다.
train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=100, shuffle=True)

# 모델을 컴파일합니다.
network = torch.compile(network)

# 신경망을 훈련합니다.
epochs = 5
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        # 매 에포크 종료 후 모델을 저장합니다.
        torch.save(
            {
                'epoch': epoch,
                'model_state_dict': network.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
            },
            "model.pt"
        )
    print("에포크:", epoch+1, "\t손실:", loss.item())

## 21.13 신경망 튜닝하기

In [ ]:
# 라이브러리를 임포트합니다.
from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import RMSprop
from torch.utils.data import random_split, DataLoader, TensorDataset
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# 훈련 세트와 테스트 세트를 만듭니다.
features, target = make_classification(n_classes=2, n_features=10,
    n_samples=1000)
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.1, random_state=1)

# 랜덤 시드를 지정합니다.
torch.manual_seed(0)
np.random.seed(0)

# 데이터를 파이토치 텐서로 변환합니다.
x_train = torch.from_numpy(features_train).float()
y_train = torch.from_numpy(target_train).float().view(-1, 1)
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(target_test).float().view(-1, 1)

# Sequential 클래스를 사용해 신경망을 정의합니다.
class SimpleNeuralNet(nn.Module):
    def __init__(self, layer_size_1=10, layer_size_2=10):
        super(SimpleNeuralNet, self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(10, layer_size_1),
            torch.nn.ReLU(),
            torch.nn.Linear(layer_size_1, layer_size_2),
            torch.nn.ReLU(),
            torch.nn.Linear(layer_size_2, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.sequential(x)
        return x

config = {
    "layer_size_1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "layer_size_2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
}

scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=1000,
    grace_period=1,
    reduction_factor=2
)

reporter = CLIReporter(
    parameter_columns=["layer_size_1", "layer_size_2", "lr"],
    metric_columns=["loss"]
)

# 신경망을 훈련합니다.
def train_model(config, epochs=3):
    network = SimpleNeuralNet(config["layer_size_1"], config["layer_size_2"])

    criterion = nn.BCELoss()
    optimizer = optim.SGD(network.parameters(), lr=config["lr"], momentum=0.9)

    train_data = TensorDataset(x_train, y_train)
    train_loader = DataLoader(train_data, batch_size=100, shuffle=True)

    # 모델을 컴파일합니다.
    network = torch.compile(network)

    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = network(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            tune.report(loss=(loss.item()))

result = tune.run(
    train_model,
    resources_per_trial={"cpu": 2},
    config=config,
    num_samples=1,
    scheduler=scheduler,
    progress_reporter=reporter
)

best_trial = result.get_best_trial("loss", "min", "last")
print("최상의 구성: {}".format(best_trial.config))
print("최상의 구성에서 최종 검증 손실: {}".format(
    best_trial.last_result["loss"]))

best_trained_model = SimpleNeuralNet(best_trial.config["layer_size_1"],
    best_trial.config["layer_size_2"]) 

## 21.14 신경망 시각화하기

In [ ]:
# 라이브러리를 임포트합니다.
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import RMSprop
from torchviz import make_dot
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# 훈련 세트와 테스트 세트를 만듭니다.
features, target = make_classification(n_classes=2, n_features=10,
    n_samples=1000)
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.1, random_state=1)

# 랜덤 시드를 지정합니다.
torch.manual_seed(0)
np.random.seed(0)

# 데이터를 파이토치 텐서로 변환합니다.
x_train = torch.from_numpy(features_train).float()
y_train = torch.from_numpy(target_train).float().view(-1, 1)
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(target_test).float().view(-1, 1)

# Sequential 클래스를 사용해 신경망을 정의합니다.
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet, self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(10, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.sequential(x)
        return x

# 신경망을 초기화합니다.
network = SimpleNeuralNet()

# 손실 함수와 옵티마이저를 정의합니다.
criterion = nn.BCELoss()
optimizer = RMSprop(network.parameters())

# 데이터 로더를 정의합니다.
train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=100, shuffle=True)

# 모델을 컴파일합니다.
network = torch.compile(network)

# 신경망을 훈련합니다.
epochs = 3
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

make_dot(output.detach(), params=dict(
    list(
        network.named_parameters()
        )
      )
    ).render(
        "simple_neural_network",
        format="png"
)